# Stimuli and session template setup example notebook
This notebook provides a minimal example on how to set up the stimuli and upload the sequence of trials to the database.

In [ ]:
from upload_to_s3 import upload_stim_to_s3
from experiment_config import experiment_file_setup

## Provide metadata and locations of the stimuli files
for a simple data directory with all to-be-uploaded files in one directory,  data_path is in the form /path/to/your/data
    
For a multi-level directory structure, you will need to use glob ** notation in data_path to index all the relevant files. something like:
- `/path/to/your/files/**/*` (this finds all the files in your directory structure)
- `/path/to/your/files/**/another_dir/*` (this finds all the files contained in all sub-directories named `another_dir`)
- `/path/to/your/files/**/another_dir/*png` (this finds all the pngs contained in all sub-directories named `another_dir`)

`bucket`: string, name of bucket to write to. Also specifies the name of the experiment in the input database.\
`pth_to_s3_credentials`: string, path to AWS credentials file\
`data_root`: string, root path for data to upload\
`data_path`: string, path in data_root to be included in upload\
`multilevel`: True for multilevel directory structures, False if all data is stored in one directory
`fam_trial_ids`: list of strings, stim_id for familiarization stimuli\
`batch_set_size`: int, # of stimuli to be included in each batch. should be a multiple of overall stimulus set size

In [ ]:
bucket = 'cognitive-ai-benchmarking-physion-stim' # bucket name on AWS S3 where stimuli where be stored
pth_to_s3_credentials = '' # local path to your aws credentials
data_root = 'Dominoes/' # make sure to have trailing / for data_root
data_path = '**/*' # this finds all subdirectories in data_root and loads all files in each subdirectory to s3
multilevel=True # Dominoes/ contains 2 subdirectories, so the structure is multi-level
s3_stim_paths = ['maps/*_map.png', 'mp4s/*_img.mp4'] # list of paths to stimuli to upload to s3—include a pattern to match only for relevant files
meta_file = './metadata.json' # path to metadata for stimulus set
fam_trial_ids = ['pilot_dominoes_0mid_d3chairs_o1plants_tdwroom_0013', 
                 'pilot_dominoes_1mid_J025R45_boxroom_0020'] # image ids for familiarization trials
batch_set_size = 37 # # 150 total stimuli, 2 familiarization trials, 4 batches

## Upload stimuli to S3
We need to store the stimuli files in S3. This assumes that a bucket has already been created and the appropriate permissions have been set (the files need to be publicly available, as they are embedded by the web experiment.) 

Make sure that you have the appropriate credentials to upload to S3. 

Running this section will upload your stimuli files to the specified S3 bucket.

In [ ]:
# upload dataset to aws s3
upload_stim_to_s3(bucket, 
                  pth_to_s3_credentials, 
                  data_root, 
                  data_path, 
                  multilevel)

## Create and upload session templates to the `input` database
This section will create a number of session templates, and upload them to the `input` database.

A session template is an ordered list of stimuli that will be shown to the participant. 

Make sure that you have appropriate credentials for the `input` database (see the documentation on the CAB config file). If you are not running this one the same machine as the database, you might need to create an ssh tunnel to the database server. (eg. run `ssh -fNL 27017:127.0.0.1:27017 USERNAME@cogtoolslab.org` in your terminal.)

In [ ]:
# batch dataset and upload to mongodb
experiment_file_setup(meta_file,
                      bucket,
                      s3_stim_paths,
                      fam_trial_ids, 
                      batch_set_size)